# Create a text import job

You use a text import job to import data from a plain text file with an accompanying import
template.

This example shows how to create a text import job request, submit it to the job
queue, and interact with the resulting text import job object returned by the server.

Information on how to create a text import template is outside the scope of this example.
For information on how to import plain text data into Granta MI, see the Granta MI documentation
or consult your ACE representative.

## Connect to Granta MI

Import the ``Connection`` class and create the connection. For more information,
see the [Connect and access the job queue](0_Getting_started.ipynb) example.

In [1]:
from ansys.grantami.jobqueue import Connection

server_url = "http://my_grantami_server/mi_servicelayer"
client = Connection(server_url).with_credentials("user_name", "password").connect()

## Create a ``TextImportJobRequest`` object

The first step in importing a text file with the job queue is to create a
``TextImportJobRequest`` object. When creating this object, specify the name of the job and the
files to import. You can also specify an optional description and the scheduled execution
date, if the import should be deferred until that date and time.

A text import job requires data files, template files, and optionally additional files to be
uploaded as attachments. These can be provided as relative or absolute paths or as `pathlib.Path`
objects.

In [2]:
import pathlib

from ansys.grantami.jobqueue import TextImportJobRequest

text_import_request = TextImportJobRequest(
    name="Text Import",
    description="An example text import job",
    template_file=pathlib.Path("./assets/text_import_template.xml"),
    data_files=["./assets/example_data.txt"],
)

text_import_request

<TextImportJobRequest: name: "Text Import">

## Submit the job to the server
Next, submit the jobs to the server. There are two methods for submitting job
requests:

* ``create_job()``: Submit the job request to the server and immediately return an
  ``AsyncJob`` object in the *pending* state.
* ``create_job_and_wait()``: Submit the job request to the server and block until the job
   either completes or fails. Return an ``AsyncJob`` object in the *succeeded* or *failed* state.

This example uses the ``create_job_and_wait()`` method. For an example that shows
how to create and submit a job that runs asynchronously, see
[Schedule and modify jobs](4_Scheduling_and_modifying_jobs.ipynb).

In [3]:
text_import_job = client.create_job_and_wait(text_import_request)
text_import_job

<AsyncJob: name: "Text Import", status: "JobStatus.Succeeded">

## Access output files
Finally, access the results of the job. Import jobs typically create log files, but the exact type
of files generated varies based on the type of import template. In this case, the files are all
plain text.

Access the list of files generated by the job with the ``output_file_names`` property. This
returns a list of file names.

In [4]:
text_import_job.output_file_names

['summary.json', 'Text Import.log']

In general, a text import job returns two files:

- ``<job name>.log``: Log file of the import operation on the server
- ``summary.json``: Data file that summarizes the number of records impacted by the import
   job and provides details of any errors that occurred during processing.

This cell shows how to access the content of the log file as ``bytes`` using the
``AsyncJob.get_file_content()`` method:

In [5]:
log_file_name = next(name for name in text_import_job.output_file_names if "log" in name)
log_file_content = text_import_job.get_file_content(log_file_name)
log_file_string = log_file_content.decode("utf-8")
print(f"{log_file_name} (first 200 characters):")
print(f"{log_file_string[:500]}...")

Text Import.log (first 200 characters):
﻿2024-12-13 10:10:03,905 [213] INFO  Task started: Template:'text_import_template.xml', Step:'', Datafile:'', Culture:''
2024-12-13 10:10:03,905 [213] INFO  Starting import process to import 4 data sets
2024-12-13 10:10:03,905 [188] INFO  Started import of Record item File set 'example_data', record number 1. for data group example_data.
2024-12-13 10:10:03,920 [188] INFO  Item Data from record example_data was processed
2024-12-13 10:10:03,998 [188] INFO  Changes committed.
2024-12-13 10:1...


This next cell shows how to download the import summary file to disk using the
``AsyncJob.download_file()`` method.

In [6]:
summary_file_name = next(
    name for name in text_import_job.output_file_names if name == "summary.json"
)
output_path = f"./{summary_file_name}"
text_import_job.download_file(summary_file_name, output_path)
print(f"{summary_file_name} saved to disk")

summary.json saved to disk
